<a href="https://colab.research.google.com/github/DaniellaMarchevska05/ml-projects/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv("insurance.csv")

# Step 1: Preprocessing
# Convert categorical data to numeric
dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})
dataset['smoker'] = dataset['smoker'].map({'yes': 1, 'no': 0})
dataset = pd.get_dummies(dataset, columns=['region'])
dataset = dataset.astype(int)

# Step 2: Split the dataset into training and testing sets
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

# Pop off the "expenses" column to create labels
train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")

# Normalize the datasets
train_stats = train_dataset.describe().transpose()
def normalize(x):
    return (x - train_stats["mean"]) / train_stats["std"]

train_dataset = normalize(train_dataset)
test_dataset = normalize(test_dataset)

# Step 3: Build the model
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation="relu", input_shape=[len(train_dataset.keys())]),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.0005),
        loss="mse",
        metrics=["mae", "mse"]
    )
    return model

model = build_model()

# Step 4: Train the model
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
history = model.fit(
    train_dataset,
    train_labels,
    epochs=200,
    validation_split=0.2,
    verbose=0,
    callbacks=[tfdocs.modeling.EpochDots(), early_stop]
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
